In [1]:
import pandas as pd

URL = 'https://raw.githubusercontent.com/ricardo-jr37/mlflow_tutorial/master/data/wine.data'


df = pd.read_csv(URL)

cols = ['label']

cols.extend(['feature_'+str(i) for i in range(1,14)])

df.columns = cols

df

,label,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13
0,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
1,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
2,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
3,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
4,1,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
173,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
174,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
175,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder




X = df.iloc[:, :-1]
y = df['label']


le = LabelEncoder()
le.fit(y)
y = le.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.3)


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)


In [3]:
import mlflow

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [14]:
pip install mlflow

Note: you may need to restart the kernel to use updated packages.


In [16]:
mlflow server --backend-store-uri sqlite:///mlrunsdb.db --default-artifact-root ./mlflowruns --host 0.0.0.0 --port

SyntaxError: invalid syntax (3676077160.py, line 1)

In [5]:
mlflow db upgrade sqlite:///mlrunsdb.db

SyntaxError: invalid syntax (4273265461.py, line 1)

In [4]:
DB_URI = 'sqlite:///mlrunsdb.db'
mlflow.set_tracking_uri(DB_URI)
tags = {
    "Módulo":"Modelos Produtivos 4",
    "Turma":815,
    "objeto":'pinguins'
}
mlflow.set_experiment(experiment_name='Classificação de especies de pinguins')
mlflow.set_experiment_tags(tags=tags)

2022/11/15 10:17:23 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2022/11/15 10:17:23 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

with mlflow.start_run(
    run_name='Tentativa 1',
    description='Classificando de vinhos',
    tags={"version":"v2"}
)as model_run:
    k = 7
    knn = KNeighborsClassifier(n_neighbors=k)

    model = Pipeline([
        ('scaler',StandardScaler()),
        ('knn',knn)
    ])

    model.fit(X_train,y_train)

In [6]:
y_pred = model.predict(X_test)

In [7]:
mlflow.log_param("k", k)

7

In [8]:
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, confusion_matrix)


def get_metrics(y_test:list, y_pred:list) -> list:
    ac = accuracy_score(y_test, y_pred)
    pr = precision_score(y_test, y_pred, average='weighted')
    rc = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return [ac, pr, rc, f1]

In [9]:
ac, pr, rc, f1 = get_metrics(y_test, y_pred)
params = {
    "k":k,
    "size_train_dataset": len(X_train),
    "size_test_dataset": len(X_test),
}

metrics = {
    "acuracia":ac,
    "precision":pr,
    "recall":rc,
    "f1":f1
}

mlflow.log_params(params=params)
mlflow.log_metrics(metrics=metrics)

mlflow.sklearn.log_model(model, "model")
print(ac, pr, rc, f1)

0.9444444444444444 0.9503086419753086 0.9444444444444444 0.9442976168756138


In [10]:
import joblib


file_name = 'knn_model_v1.pkl'


joblib.dump(model, file_name)

['knn_model_v1.pkl']

In [11]:
val_model = joblib.load('knn_model_v1.pkl')

val_model.predict(X_val)

array([0, 1, 1, 2, 2, 0, 0, 1, 0, 2, 2, 1, 0, 0, 0, 1, 2, 1, 2, 1, 0, 2,
       0, 0, 2, 1, 1, 0, 2, 0, 0])

In [12]:
val_model.score(X_val, y_val)

0.9354838709677419

In [13]:
import mlflow

mlflow.set_tracking_uri(uri='http://localhost:5000/')
mlflow.sklearn.load_model('models:/classificacao_vinho/Production')

WARNI [urllib3.connectionpool] Retrying (Retry(total=4, connect=4, read=5, redirect=5, status=5)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7effcb55e160>: Failed to establish a new connection: [Errno 111] Connection refused')': /api/2.0/mlflow/registered-models/get-latest-versions
WARNI [urllib3.connectionpool] Retrying (Retry(total=3, connect=3, read=5, redirect=5, status=5)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7effcb55e430>: Failed to establish a new connection: [Errno 111] Connection refused')': /api/2.0/mlflow/registered-models/get-latest-versions
WARNI [urllib3.connectionpool] Retrying (Retry(total=2, connect=2, read=5, redirect=5, status=5)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7effcb55e610>: Failed to establish a new connection: [Errno 111] Connection refused')': /api/2.0/mlflow/registered-models/get-latest-vers

MlflowException: API request to http://localhost:5000/api/2.0/mlflow/registered-models/get-latest-versions failed with exception HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/2.0/mlflow/registered-models/get-latest-versions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7effcb552dc0>: Failed to establish a new connection: [Errno 111] Connection refused'))